# Analýza věku a pohlaví podle pětiletých věkových skupin

### Import knihoven


In [1]:
import pandas as pd
import numpy as np
import pymongo

### Poznámky

In [ ]:
# https://www.czso.cz/csu/rso/uzemne_technicke_jednotky
# https://scitani.gov.cz/vysledky
# https://www.czso.cz/csu/czso/vysledky-scitani-2021-otevrena-data

### Pomocné funkce

In [11]:
# Tisk čísla s oddělovači čárek
def format_with_commas(number):
    return "{:,}".format(number)

# Hezký tisk slovníku pro věk
def print_dictionary_ages(text, dict):
    print(f"{text}")
    for key, value in dict.items():
        print(f"\t=> {key}: {value} ({(value / analysis_result['count'])*100:.2f}%)")

# Hezký tisk slovníku pro věk a pohlaví
def print_dictionary_ages_sex(text, dict):
    print(f"{text}")
    for key, value in dict.items():
        print(f"\t=> {key}:")
        for key, value in value.items():
            print(f"\t\t=> {key}: {value}")

### Načtení dat do paměti z MongoDB

In [3]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["scitani_lidu"]
collection = db["obyvatelstvo_pohlavi_vek_5"]

data = []

for index, document in enumerate(collection.find()):
    data.append(document)

data = pd.DataFrame(data)
data = data.replace({np.nan: None})

### Analýza

In [18]:
# Pomocné CIS kódy
cis_vek = {
    1100000004: '0 - 4 roky',
    1300050009: '5 - 9 let',
    1300100014: '10 - 14 let',
    1300150019: '15 - 19 let',
    1300200024: '20 - 24 let',
    1300250029: '25 - 29 let',
    1300300034: '30 - 34 let',
    1300350039: '35 - 39 let',
    1300400044: '40 - 44 let',
    1300450049: '45 - 49 let',
    1300500054: '50 - 54 let',
    1300550059: '55 - 59 let',
    1300600064: '60 - 64 let',
    1300650069: '65 - 69 let',
    1300700074: '70 - 74 let',
    1300750079: '75 - 79 let',
    1300800084: '80 - 84 let',
    1300850089: '85 - 89 let',
    1300900094: '90 - 94 let',
    1300950099: '95 - 99 let',
    1201009999: '100 a více let'
}

veky_podle_fazi = {
    'deti': ['0 - 4 roky', '5 - 9 let'],
    'dospivani': ['10 - 14 let','15 - 19 let'],
    'dospeli': ['20 - 24 let', '25 - 29 let', '30 - 34 let', '35 - 39 let', '40 - 44 let', '45 - 49 let', '50 - 54 let', '55 - 59 let'],
    'stari': ['65 - 69 let', '70 - 74 let', '75 - 79 let', '80 - 84 let', '85 - 89 let', '95 - 99 let', '100 a více let'],
}

In [23]:
analysis_result = dict()

# Celkový počet obyvatel
analysis_result['count'] = int(data[(data['uzemi_cis'] == 97) & (data['vek_txt'].isnull()) & (data['pohlavi_txt'].isnull())]['hodnota'])

# Celkový počet obyvatel podle pohlaví
analysis_result['count_women'] = int(data[(data['uzemi_cis'] == 97) & (data['vek_txt'].isnull()) & (data['pohlavi_kod'] == 2)]['hodnota'])
analysis_result['count_men'] = int(data[(data['uzemi_cis'] == 97) & (data['vek_txt'].isnull()) & (data['pohlavi_kod'] == 1)]['hodnota'])

# Celkový počet obyvatel podle věku
analysis_result['count_ages'] = {}
for key, value in cis_vek.items():
    analysis_result['count_ages'][value] = int(data[(data['uzemi_cis'] == 97) & (data['vek_kod'] == key) & (data['pohlavi_txt'].isnull())]['hodnota'])

# Celkový počet obyvatel podle věku a pohlaví
analysis_result['count_age_sex'] = {}
for key, value in cis_vek.items():
    analysis_result['count_age_sex'][value] = {}
    analysis_result['count_age_sex'][value]['women'] = int(data[(data['uzemi_cis'] == 97) & (data['vek_kod'] == key) & (data['pohlavi_kod'] == 2)]['hodnota'])
    analysis_result['count_age_sex'][value]['men'] = int(data[(data['uzemi_cis'] == 97) & (data['vek_kod'] == key) & (data['pohlavi_kod'] == 1)]['hodnota'])

# Celkový počet obyvatel podle věkových fází (období)
analysis_result['count_age_phase'] = {}
for key, value in veky_podle_fazi.items():
    analysis_result['count_age_phase'][key] = 0
    for phase in value:
        analysis_result['count_age_phase'][key] += int(data[(data['uzemi_cis'] == 97) & (data['vek_txt'] == phase) & (data['pohlavi_txt'].isnull())]['hodnota'])

# Celkový počet obyvatel podle věkových fází (období) a pohlaví
analysis_result['count_age_phase_sex'] = {}
for key, value in veky_podle_fazi.items():
    analysis_result['count_age_phase_sex'][key] = {}
    analysis_result['count_age_phase_sex'][key]['women'] = 0
    analysis_result['count_age_phase_sex'][key]['men'] = 0
    for phase in value:
        analysis_result['count_age_phase_sex'][key]['women'] += int(data[(data['uzemi_cis'] == 97) & (data['vek_txt'] == phase) & (data['pohlavi_kod'] == 2)]['hodnota'])
        analysis_result['count_age_phase_sex'][key]['men'] += int(data[(data['uzemi_cis'] == 97) & (data['vek_txt'] == phase) & (data['pohlavi_kod'] == 1)]['hodnota'])

### Výsledky analýzy

In [25]:
# Celkový počet obyvatel
# print(f"Celkový počet obyvatel: {format_with_commas(analysis_result['count'])}")
# print(f"\nCelkový počet žen: {format_with_commas(analysis_result['count_women'])} ({(analysis_result['count_women'] / analysis_result['count'])*100:.2f}%)")
# print(f"\nCelkový počet muž: {format_with_commas(analysis_result['count_men'])} ({(analysis_result['count_men'] / analysis_result['count'])*100:.2f}%)")
# print_dictionary_ages("\nCelkový počet obyvatel podle věku:", analysis_result['count_ages'])
# print_dictionary_ages_sex("\nCelkový počet obyvatel podle věku a pohlaví:", analysis_result['count_age_sex'])
print_dictionary_ages("\nCelkový počet obyvatel podle věkoých fází:", analysis_result['count_age_phase'])
print_dictionary_ages_sex("\nCelkový počet obyvatel podle věkových fází a pohlaví:", analysis_result['count_age_phase_sex'])


Celkový počet obyvatel podle věku a pohlaví:
	=> 0 - 4 roky:
		=> women: 273743
		=> men: 286889
	=> 5 - 9 let:
		=> women: 267952
		=> men: 282072
	=> 10 - 14 let:
		=> women: 283743
		=> men: 297361
	=> 15 - 19 let:
		=> women: 238290
		=> men: 251546
	=> 20 - 24 let:
		=> women: 231724
		=> men: 245339
	=> 25 - 29 let:
		=> women: 290569
		=> men: 310599
	=> 30 - 34 let:
		=> women: 333508
		=> men: 357053
	=> 35 - 39 let:
		=> women: 347389
		=> men: 371700
	=> 40 - 44 let:
		=> women: 414631
		=> men: 443286
	=> 45 - 49 let:
		=> women: 426913
		=> men: 450703
	=> 50 - 54 let:
		=> women: 337187
		=> men: 350923
	=> 55 - 59 let:
		=> women: 331655
		=> men: 334736
	=> 60 - 64 let:
		=> women: 315642
		=> men: 300966
	=> 65 - 69 let:
		=> women: 356534
		=> men: 311547
	=> 70 - 74 let:
		=> women: 345900
		=> men: 272729
	=> 75 - 79 let:
		=> women: 249134
		=> men: 170234
	=> 80 - 84 let:
		=> women: 153868
		=> men: 89505
	=> 85 - 89 let:
		=> women: 92099
		=> men: 43212
	=> 90

AttributeError: 'int' object has no attribute 'items'